## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests

import os
sys.path.append("..")
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Berlevag,NO,2022-07-20 20:38:43,70.8578,29.0864,55.20,94,100,2.30,overcast clouds
1,1,Wuchang,CN,2022-07-20 20:38:43,44.9143,127.1500,67.19,94,97,3.87,overcast clouds
2,2,Makakilo City,US,2022-07-20 20:38:43,21.3469,-158.0858,80.69,69,40,13.80,scattered clouds
3,3,Ushuaia,AR,2022-07-20 20:38:44,-54.8000,-68.3000,33.42,93,100,21.85,snow
4,4,Bethel,US,2022-07-20 20:35:49,41.3712,-73.4140,98.02,50,20,8.05,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# setting min at 65 and max at 75 for running this

min_temp = float(input("What's the coldest temperature (F) you'd like? "))
max_temp = float(input("What's the warmest temperature (F) you'd like? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temperate_df = city_data_df[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
temperate_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Long                   0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = temperate_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Long"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Long,Hotel Name
1,Wuchang,CN,67.19,overcast clouds,44.9143,127.1500,
15,Royan,FR,67.84,clear sky,45.6285,-1.0281,
24,Nikolskoye,RU,69.87,overcast clouds,59.7035,30.7861,
28,Flin Flon,CA,67.84,broken clouds,54.7682,-101.8650,
29,Rikitea,PF,73.31,clear sky,-23.1203,-134.9692,
31,Sasciori,RO,70.11,clear sky,45.8667,23.5833,
33,Thunder Bay,CA,71.58,overcast clouds,48.4001,-89.3168,
34,Waingapu,ID,70.63,few clouds,-9.6567,120.2641,
47,Port Keats,AU,66.74,overcast clouds,-14.2500,129.5500,
59,Saint-Philippe,RE,69.39,scattered clouds,-21.3585,55.7679,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    long = row["Long"]
    params["location"] = f"{lat},{long}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        #just pick the first hotel from the results
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        no_hotel_city = hotel_df.loc[index, "City"]
        print(f"Hotel in {no_hotel_city} not found...skipping")
        

Hotel in Ndele not found...skipping
Hotel in Businga not found...skipping
Hotel in Lumeje not found...skipping
Hotel in Ganta not found...skipping
Hotel in Baoro not found...skipping
Hotel in Carnot not found...skipping
Hotel in Mokhsogollokh not found...skipping
Hotel in Salvacion not found...skipping
Hotel in Shyryayeve not found...skipping
Hotel in Caxito not found...skipping
Hotel in Konstantinovka not found...skipping
Hotel in Mounana not found...skipping
Hotel in Zhelyabovka not found...skipping
Hotel in Angoram not found...skipping
Hotel in Mitu not found...skipping


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]

In [19]:
# 8. Export the City_Data into a csv
hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Long"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
#build basemap
fig = gmaps.figure(center=(30.0,31.0), zoom_level=2.0)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))